**Learning outcomes**
* Define an observable, and compute its possible measurement outcomes.
* Compute the expectation value of an observable.

Rather than simply measuring outcome probabilities, when solving physical problems, we need too compute a measurable quantity that relates to some property os system (i.e. energy, momentum).Such properties are called **observables**. Mathematically, observables correspond to Hermitian matrics, i.e., matrices $B$ such that $B=B^*$. Each observable has some set of possible measurement outcomes, corresponding to their real eigenvalues. HOwever, since measurement is probabilistic, we usually want to look for the **expectation value**, demoted by $<B>$, of that physical property.

It is very straightforward to compute expectation values in PennyLane. We can simplify replace the `qml.probs` of the previous sections with `qml.expval`, and specify the observables to be measured. Common choices are `qml.PauliX, qml.PauliY, qml.PauliZ` as many algorithms that solve physical problems require computing the expectation values of the Pauli operators. The possible outcomes of measuring any Pauli-based expectation value are either $1$ or $-1$, as these are their eigenvalues.
These three Pauli operators are unitary and Hermitian, so can be used as gates in circuit, and as observables.

[qml.probs documentation](https://docs.pennylane.ai/en/stable/code/api/pennylane.probs.html)
[qml.expval documentation](https://docs.pennylane.ai/en/stable/code/api/pennylane.expval.html)

In [1]:
import numpy as np
import pennylane as qml

**Codercise I.10.1**
Design and run a PennyLane circuit that performs the following, where $<Y>$ indicates measurement of the `PauliY` observable.
![circuit](./images/I.10.1.png)

In [2]:
dev = qml.device('default.qubit', wires=1)

@qml.qnode(dev)
def circuit():
    # IMPLEMENT THE CIRCUIT IN THE PICTURE AND MEASURE PAULI Y
    qml.RX(np.pi/4, wires=0)
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=0)

    return qml.expval(qml.PauliY(0))

print(circuit())

-0.7071067811865471


You can also specify a custom observable using `qml.Hermitian`
[qml.Hermitian](https://docs.pennylane.ai/en/stable/code/api/pennylane.Hermitian.html)
`#single-qubit Hemitian observable`
`Obs = np.array([[3,4i],[-4i,3]])`
`@qml.qnode(dev)`
`def circuit():`
`...`
`return qml.expval(qml.Hermitian(Obs, wires=0))`

In the previous sections, we computed measurement outcome probabilities and expectation values analytically. Of course, this is impossible to do on hardware. When we run a circuit and take a measurement, we get a single data point that tells us in which state we observed a qubit for a particular run. Since this process is random, in order to get a clearer picture of the statistics we must perform the experiment many, many times. Each time is typically called a shot, or a sample. We can sample from the output distribution in order to estimate expectation values and measurement outcome probabilities in situations where it isn't possible to do so analytically.
In PennyLane, the number of shots to take during an experiment is specified upon device construction
`dev = qml.device('default.qubit', wires=1, shots=1000)`

**CoderciseI.10.2.**
In the code below is a list of possible numbers of shots. For each value, initialize a device, then create and QNode that runs circuit from the previous exercise (reproduced below for convenience). What happens to the expectation value as you increase the number shots?
[qml.math.unwrap](https://docs.pennylane.ai/en/stable/code/api/pennylane.math.unwrap.html)

In [8]:
# An array to store your results
shot_results = []

# Different numbers of shots
shot_values = [100, 1000, 10000, 100000, 1000000]

for shots in shot_values:
# CREATE A DEVICE, CREATE A QNODE, AND RUN IT
# STORE RESULT IN SHOT_RESULTS ARRAY
    dev = qml.device('default.qubit', wires = 1, shots =shots)
    shot_results.append(circuit())
    print(shot_results[-1])



print(qml.math.unwrap(shot_results))


-0.707826
-0.706678
-0.706304
-0.70585
-0.70684
[-0.707826, -0.706678, -0.706304, -0.70585, -0.70684]


What happens under the hood in the exercise above is that the circuit is run multiple times, and a measurement is made, yielding on of the two eigenvalues($1$ aor $-1$). We can use the result of these samples to compute the expectation value in the same way as we would normally take a `weighted average`,i.e.,
$$<Y> = 1*(number of 1s)+(-1)*(number of -1s)\over number of shots$$

In PennyLane, we can access samples directly by returning `qml.sample` rather than `qml.expval`.

**Codercise I.10.3.** Using the circuit from earlier, replace the `qml.expval` measurement with `qml.sample.` Then, write a function to compute an estimate of the expectation value based on the samples.

In [9]:
dev = qml.device("default.qubit", wires=1, shots=100000)

@qml.qnode(dev)
def circuit():
    qml.RX(np.pi/4, wires=0)
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=0)

    # RETURN THE MEASUREMENT SAMPLES OF THE CORRECT OBSERVABLE
    return qml.sample(qml.PauliY(0))


def compute_expval_from_samples(samples):
    """Compute the expectation value of an observable given a set of
    sample outputs. You can assume that there are two possible outcomes,
    1 and -1.

    Args:
        samples (array[float]): 100000 samples representing the results of
            running the above circuit.

    Returns:
        float: the expectation value computed based on samples.
    """

    estimated_expval = 0
    number_shots = 100000
    eigenvalue_one = 1
    eigenvalue_two = -1

    # USE THE SAMPLES TO ESTIMATE THE EXPECTATION VALUE
    count_eigenvalue_one = np.count_nonzero(samples == eigenvalue_one)
    count_eigenvalue_two = np.count_nonzero(samples == eigenvalue_two)
    estimated_expval = (eigenvalue_one*count_eigenvalue_one + eigenvalue_two*count_eigenvalue_two)/number_shots

    return estimated_expval


samples = circuit()
print(compute_expval_from_samples(samples))

-0.7046


**Codercise I.10.4**
Using the code below as a staring point, explore how the accuracy of the expectation value depends on the number of shots.For example, if we run 100 experiments with 100 shots each, what are the mean and variance of the distribution od expectation values obtained? HOw does this variance scale with the number of sots?

To accomplish this, we will use a very simple circuit that consists of a Hadamard and a measurement of the `qml.PauliZ` observable, which will allow us to directly extract the dependence of variance on the number of shots. We will plot your result; based on the plot, complete the `variance_scaling` function below to define the relationship between variance and shots.

In [11]:
def variance_experiment(n_shots):
    """Run an experiment to determine the variance in an expectation
    value computed with a given number of shots.

    Args:
        n_shots (int): The number of shots

    Returns:
        float: The variance in expectation value we obtain running the
        circuit 100 times with n_shots shots each.
    """

    # To obtain a variance, we run the circuit multiple times at each shot value.
    n_trials = 100
    array_expvals = []


    # CREATE A DEVICE WITH GIVEN NUMBER OF SHOTS
    dev = qml.device('default.qubit', wires=1, shots = n_shots)
    # DECORATE THE CIRCUIT BELOW TO CREATE A QNODE
    @qml.qnode(dev)
    def circuit():
        qml.Hadamard(wires=0)
        return qml.expval(qml.PauliZ(wires=0))

    # RUN THE QNODE N_TRIALS TIMES AND RETURN THE VARIANCE OF THE RESULTS
    array_expvals.append(circuit() for i in n_trials)

   #mean = np.mean(array_expval)
    variance = np.var(array_expvals)

    return variance


def variance_scaling(n_shots):
    """Once you have determined how the variance in expectation value scales
    with the number of shots, complete this function to programmatically
    represent the relationship.

    Args:
        n_shots (int): The number of shots

    Returns:
        float: The variance in expectation value we expect to see when we run
        an experiment with n_shots shots.
    """

    estimated_variance = 0
    eigenvalue_one = 1
    eigenvalue_two = -1

    # ESTIMATE THE VARIANCE BASED ON SHOT NUMBER
    # CREATE A DEVICE WITH GIVEN NUMBER OF SHOTS
    dev = qml.device('default.qubit', wires=1, shots = n_shots)
    # DECORATE THE CIRCUIT BELOW TO CREATE A QNODE
    @qml.qnode(dev)
    def circuit():
        qml.Hadamard(wires=0)
        return qml.sample(qml.PauliZ(wires=0))
    samples = circuit()
    count_eigenvalues_one = np.count_nonzero(samples == eigenvalue_one)
    count_eigenvalues_two = np.count_nonzero(samples == eigenvalue_two)
    estimated_variance = (eigenvalue_one * count_eigenvalues_one + eigenvalue_two * count_eigenvalues_two)/n_shots
    return estimated_variance

# Various numbers of shots; you can change this
shot_vals = [10, 20, 40, 100, 200, 400, 1000, 2000, 4000]

# Used to plot your results
#results_experiment = [variance_experiment(shots) for shots in shot_vals]
results_scaling = [variance_scaling(shots) for shots in shot_vals]
#plot = plotter(shot_vals, results_experiment, results_scaling)